In [1]:
import sys
sys.path.append("..")


In [2]:
from data.dataloader import LowLightDataModule
from model.blocks.contourlet import LaplacianPyramid, DirectionalFilterBank, ContourletTransform
from utils.utils import show_batch


In [3]:
data_module = LowLightDataModule(
    train_dir="../data/1_train",
    valid_dir="../data/2_valid",
    bench_dir="../data/3_bench",
    infer_dir="../data/4_infer",
    image_size=640,
    batch_size=32,
    num_workers=4
)

data_module.setup()


In [4]:
train_dataloader = data_module.train_dataloader()


In [ ]:
train_low, train_high = next(iter(train_dataloader))
print(train_low.shape)
print(train_high.shape)
# show_batch(images=train_low)
# show_batch(images=train_high)


torch.Size([32, 3, 640, 960])
torch.Size([32, 3, 640, 960])


In [6]:
lp  = LaplacianPyramid(
    in_channels=3,
    num_levels=4,
    filter_size=5,
    sigma=1.0
)

py, lc = lp(train_low)


c:\Users\jungih\dev\ContourDiffusion\.venv\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
print(len(py), len(lc))


4 4


In [8]:
for i, (p, l) in enumerate(zip(py, lc)):
    print("--------")
    print("p shape :", p.shape)
    print("l shape :", l.shape)
    # show_batch(images=p)
    # show_batch(images=l)


--------
p shape : torch.Size([32, 3, 640, 960])
l shape : torch.Size([32, 3, 640, 960])
--------
p shape : torch.Size([32, 3, 320, 480])
l shape : torch.Size([32, 3, 320, 480])
--------
p shape : torch.Size([32, 3, 160, 240])
l shape : torch.Size([32, 3, 160, 240])
--------
p shape : torch.Size([32, 3, 80, 120])
l shape : torch.Size([32, 3, 80, 120])


In [9]:
dfb_1 = DirectionalFilterBank(
    in_channels=3,
    num_levels=1,
    filter_size=5,
    sigma=1.0,
    omega_x=0.25,
    omega_y=0.25
)

dfb_4 = DirectionalFilterBank(
    in_channels=3,
    num_levels=4,
    filter_size=5,
    sigma=1.0,
    omega_x=0.25,
    omega_y=0.25
)

subband_1 = dfb_1(lc[0])
print("--------", len(subband_1))
for s in subband_1:
    print("s shape :", s.shape)
    # show_batch(images=s)

subband_4 = dfb_4(lc[-1])
print("--------", len(subband_4))
for s in subband_4:
    print("s shape :", s.shape)
    # show_batch(images=s)


-------- 2
s shape : torch.Size([32, 3, 640, 960])
s shape : torch.Size([32, 3, 640, 960])
-------- 16
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])


In [10]:
contourlet = ContourletTransform(
    in_channels=3,
    num_levels=4,
    filter_size=5,
    sigma=1.0,
    omega_x=0.25,
    omega_y=0.25
)

pyramid, subbands = contourlet(train_low)


In [11]:
for i, p in enumerate(iterable=pyramid):
    print("--------"ArithmeticError, len(p))
    print("p shape :", p.shape)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2951637138.py, line 2)

In [ ]:
for i, subband in enumerate(iterable=subbands):
    print("--------", len(subband))
    for i, s in enumerate(iterable=subband):
        print("s shape :", s.shape)


-------- 16
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
s shape : torch.Size([32, 3, 80, 120])
-------- 8
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
s shape : torch.Size([32, 3, 160, 240])
-------- 4
s shape : torch.Size([